In [ ]:
import os
from pathlib import Path
import sys
sys.path.append('../src')

import numpy as np
from matplotlib import pyplot as plt

from nlkda.data.loader import get_data, DatasetEnum
from nlkda.models.base import MonotonicityWrapper
from nlkda.models.utils import load_model_from_directory
from nlkda.settings import K_MAX

In [ ]:
def plot_bounds(lower, upper, lower_mono, upper_mono, real, i, f_name):
    x = np.arange(1, K_MAX+1)
    l, = plt.plot(x, lower[i,:], label='original lower', ls="dashed", color="deepskyblue", zorder=99)
    u, = plt.plot(x, upper[i,:], label='original upper', ls="dashed", color="orange", zorder=99)
    ml, = plt.plot(x, lower_mono[i,:], label='improved lower', color="blue", zorder=99)
    mu, = plt.plot(x, upper_mono[i,:], label='improved upper', color="red", zorder=99)
    plt.plot(x, real[i,:], label='Real', color="black", zorder=99, ls="dotted",)
    
    plt.fill_between(x, lower[i,:], lower_mono[i,:], color=l.get_color(), alpha=.4)
    plt.fill_between(x, upper[i,:], upper_mono[i,:], color=u.get_color(), alpha=.4)
    
    plt.xlim(1, K_MAX)
    plt.ylim(0, None)
    plt.grid(which='minor', alpha=0.4)
    plt.grid(which='major', alpha=0.5)
    plt.xlabel('k')
    plt.ylabel('distance')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'{f_name}.pdf')

In [ ]:
data_root = Path("/tmp/data/")
exp = data_root / 'experiments'

ds = "OL"
run_id = '' # TODO: fill in run_id
out_dir = exp / run_id

x, y, dist = get_data(DatasetEnum(ds), data_root)

# bounds model
model = load_model_from_directory(out_dir)
pred = np.load(os.path.join(out_dir,'pred_k_dist.npy'))

model.set_min_max(pred, y, True, True)
lower_p, upper_p = model.predict_bounds(pred, True, True)
mono_wrapper = MonotonicityWrapper(base=model)
mono_lower_p, mono_upper_p = mono_wrapper.predict_bounds(pred, True, True)  

model.set_min_max(pred, y, True, False)
lower_k, upper_k = model.predict_bounds(pred, True, False)
mono_wrapper_p = MonotonicityWrapper(base=model)
mono_lower_k, mono_upper_k = mono_wrapper_p.predict_bounds(pred, True, False)

# get best lower and best upper
lower_p_k = np.maximum(mono_lower_k, mono_lower_p)
upper_p_k = np.minimum(mono_upper_k, mono_upper_p)

In [ ]:
plot_bounds(
    lower=lower_k, 
    upper=upper_k, 
    lower_mono=mono_lower_k, 
    upper_mono=mono_upper_k, 
    real=y, 
    i=0, 
    f_name='improve_k'
)